In [ ]:
import sys; print(sys.path[:8])

import sys, torch
print("python:", sys.executable)
print("torch from:", torch.__file__)
print("torch:", torch.__version__, "| compiled cuda:", torch.version.cuda)
print("cuda available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("device:", torch.cuda.get_device_name(0))


: 

In [2]:
pip install pandas numpy


error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [2]:

import os
print("Current working directory:", os.getcwd())
import importlib

from pathlib import Path

# 脚本运行时的项目根目录：脚本文件所在目录（在 notebook 里用 Path.cwd()）
ROOT = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()

DATA_DIR = ROOT  # 你原来就把项目放在 /home/yyc/data
print("Current working directory:", ROOT)

#from torchdiffeq import odeint
import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
#from model.init import model
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from model.Multiagent import MultiAgentFusion
# GNNs (Graph Neural Network) 
from model.mlp import MLPModel
# MAMBA 
import torch
torch.cuda.empty_cache()
import torch.nn as nn
#from model.mamba import Mamba
# LSTM 
#from model.lstm import LSTM
# GRU 
from sklearn.preprocessing import StandardScaler
from model.gru import GRUModel
# Neural Network 
#from model.neural import NeuralODE
# Transformer 
#from model.transformer import Transformer
import numpy as np


import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
#os.environ['CUDA_VISIBLE_DEVICES'] = "0"
print(torch.__version__)  # 查看torch当前版本号
print(torch.version.cuda)  # 编译当前版本的torch使用的cuda版本号
print(torch.cuda.is_available())  # 查看当前cuda是否可用于当前版本的Torch
from torch.utils.data import DataLoader, Dataset

Current working directory: /home/yyc/data
Current working directory: /home/yyc/data


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
# 定义一个 Dataset 子类，负责加载数据
class LargeDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [ ]:
class Transformer(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Transformer, self).__init__()
        self.embedding = nn.Linear(input_size, hidden_size)
        self.transformer = nn.Transformer(d_model=hidden_size, num_encoder_layers=6, nhead=4)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        x = self.embedding(x)  # 输入特征进行嵌入
        x = x.unsqueeze(0)  # 加一个 batch 维度
        x = self.transformer(x, x)  # Transformer 进行处理
        x = x.squeeze(0)  # 移除 batch 维度
        x = self.fc(x)  # 输出预测
        return x

In [ ]:
data = pd.read_csv('./TrafficLabelling/end.csv').values
le = LabelEncoder()
le.fit(data[:,-1])

data[:,-1]= le.transform(data[:,-1])
num_classes = len(np.unique(data[:,-1]))


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_layer_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers, batch_first=True)

        # Fully connected layer
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        #print(f"Input shape before unsqueeze: {input_seq.shape}")  # Debug: Check input shape
        if len(input_seq.shape) == 2:
            input_seq = input_seq.unsqueeze(1)
            #print(f"Input shape after unsqueeze: {input_seq.shape}")  # Debug: Should now be 3D

        lstm_out, _ = self.lstm(input_seq)
        #print(f"LSTM output shape: {lstm_out.shape}")  # Debug: [batch_size, seq_len, hidden_layer_size]
        lstm_out_last = lstm_out[:, -1, :]
        predictions = self.linear(lstm_out_last)
        return predictions

In [ ]:
class MambaPredictor(nn.Module):
    def __init__(self, input_dim, seq_len, n_classes, d_model=16, d_state=2, d_conv=4, expand=2):
        super().__init__()
        self.input_dim = input_dim
        self.seq_len = seq_len
        self.d_model = d_model
        self.n_classes = n_classes

        self.input_embedding = nn.Linear(input_dim, d_model)
        self.mamba = Mamba(d_model=d_model, d_state=d_state, d_conv=d_conv, expand=expand, use_fast_path=False)
        self.fc = nn.Linear(d_model, n_classes)

    def forward(self, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(1)

        x = self.input_embedding(x)
        #print(f"After input_embedding: {x}")  # Debugging step
        x = self.mamba(x)
        #print(f"After Mamba: {x}")  # Debugging step
        x = x[:, -1, :]  # Take last time step
        x = self.fc(x)
        return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
X_train, X_test, y_train, y_test = train_test_split(data[:,:-1].astype('float64'), data[:,-1].astype('float64'), test_size=0.2)
class_0_indices = np.where(y_train == 0)[0]
other_classes_indices = np.where(y_train != 0)[0]

# Randomly select half of the '0' class samples
reduced_class_0_indices = np.random.choice(class_0_indices, size=len(class_0_indices) // 100, replace=False)

# Combine reduced '0' class indices with all other class indices
balanced_indices = np.concatenate([reduced_class_0_indices, other_classes_indices])
X_train[np.isinf(X_train)] = 0
X_train[np.isnan(X_train)] = 0
X_test[np.isinf(X_test)] = 0
X_test[np.isnan(X_test)] = 0


In [ ]:

# Create the new balanced dataset
X_train= X_train[balanced_indices]
y_train= y_train[balanced_indices]
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train=X_train
X_test=X_test
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4, pin_memory=True)

dimension = X_train.shape[1]
print(dimension)

In [ ]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
num_classes = len(np.unique(y_train))
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)

plt.hist(y_train, bins=np.arange(num_classes + 1) - 0.5, edgecolor='black')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Class Distribution')
plt.show()



In [ ]:

class NeuralODE(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(NeuralODE, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        # 定义一个简单的全连接网络作为ODE的右侧函数
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)

    def forward(self, t, x):
        """
        神经网络模型的ODE右侧函数
        参数：
        - t: 时间点（在ODE求解中自动传递）
        - x: 当前输入
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x  # 返回logits


    def solve(self, x0, t):
        """
        使用odeint求解ODE，得到模型的输出
        参数：
        - x0: 初始状态
        - t: 时间序列
        """
        # odeint通过自动微分求解ODE
        solution = odeint(self, x0, t)
        return solution[-1]  # 取最后时刻的输出


In [ ]:
class Agent:
    def __init__(self, model, optimizer, loss_fn, device):
        """
        初始化Agent。

        参数：
        - model: 该Agent的模型
        - optimizer: 优化器
        - loss_fn: 损失函数
        - device: 设备（'cpu' 或 'cuda'）
        """
        self.model = model # 将模型移动到指定设备
        self.optimizer = optimizer
        self.loss_fn = loss_fn
        self.reward = 0.0  # 初始化奖励值
        self.device = device
        self.weight = 1.0  # 初始化权重
     
    def train_NeuralODE(self, inputs, target):
        self.model.train()
        self.optimizer.zero_grad()
        t = torch.linspace(0, 1, steps=100).to(self.device)
        # 假设 forward 方法期望的是 'inputs' 而非 'x'
        outputs = self.model(t,inputs)  # 使用正确的参数名传入
        if target.dtype != torch.long:
            target = target.long() 
        loss = self.loss_fn(outputs, target)
        loss.backward()
        self.optimizer.step()
        return loss.item()
    def train(self, inputs, target):
        self.model.train()
        self.optimizer.zero_grad()
        # 假设 forward 方法期望的是 'inputs' 而非 'x'
        outputs = self.model(inputs)  # 使用正确的参数名传入
        if target.dtype != torch.long:
            target = target.long() 
        loss = self.loss_fn(outputs, target)
        loss.backward()
        self.optimizer.step()
        return loss.item()
    def predict_NeuralODE(self, X):
        # 获取当前设备
        device = next(self.model.parameters()).device  # 获取模型参数所在设备

        # 创建时间序列 t，并移动到相同设备
        t = torch.linspace(0, 1, steps=10, device=device)  # 假设时间点 t 需要在相同设备上

        if not isinstance(X, torch.Tensor):
            X = torch.tensor(X, dtype=torch.float32, device=device)  # 将 X 移动到同一个设备

        # 使用相同设备进行前向传播
        outputs = self.model(t, X)  # 模型前向传播
        
        return outputs.detach().cpu().numpy()  # 返回 CPU 上的 numpy 数组

    def predict(self, X):
        # 获取模型所在的设备
        device = next(self.model.parameters()).device  # 获取模型参数所在的设备

        if isinstance(X, torch.Tensor):  # 如果 X 已经是 tensor 类型
            X = X.to(device)  # 将 X 移动到模型所在的设备
        else:
            # 如果 X 是 numpy 数组，先转为 tensor，并将其移动到正确设备
            X = torch.tensor(X, dtype=torch.float32).to(device)

        # 使用与设备一致的张量进行预测
        outputs = self.model(X)  # 模型前向传播
        return outputs.detach().cpu().numpy()  # 返回 CPU 上的 numpy 数组

    def update_reward(self, reward):
        """
        更新Agent的奖励值。

        参数：
        - reward: 新的奖励值
        """
        self.reward += reward

    def adjust_learning_rate(self, min_lr=0.001, max_lr=0.01):
        """
        根据奖励动态调整学习率。

        参数：
        - min_lr: 最小学习率
        - max_lr: 最大学习率
        """
        for param_group in self.optimizer.param_groups:
            new_lr = max(min_lr, min(max_lr, param_group['lr'] + self.reward * 0.001))
            param_group['lr'] = new_lr

    def adjust_weight(self):
        """
        根据奖励值调整Agent的权重。
        """
        self.weight = max(0.1, self.reward)  # 确保最小权重为0.1

    @staticmethod
    def normalize_weights(agents):
        """
        归一化所有Agent的权重，使得权重之和为1。

        参数：
        - agents: 所有Agent的列表
        """
        total_reward = sum(agent.reward for agent in agents)
        if total_reward > 0:
            for agent in agents:
                agent.weight = agent.reward / total_reward
        else:
            for agent in agents:
                agent.weight = 1.0 / len(agents)  # 如果没有奖励，均分权重


In [ ]:
print(num_classes)

In [ ]:

#model = MultiAgentFusion(input_size=5, hidden_size=64, output_size=num_classes)
mlp_model = MLPModel(input_size=dimension, hidden_size=64, output_size=num_classes)
gru_model = GRUModel(input_size=dimension, hidden_layer_size=64, num_layers=2, output_size=num_classes)
lstm_model = LSTM(input_size=dimension, hidden_layer_size=64, num_layers=2, output_size=num_classes)
batch, length, dim = 2, 64, 768
#mamba_model = MambaPredictor(input_dim=dimension, seq_len=1, n_classes=num_classes)
neural_model = NeuralODE(input_dim=dimension, hidden_dim=1, output_dim=num_classes)
transformer_model = Transformer(input_size=dimension, hidden_size=4, num_classes=num_classes)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  

In [ ]:
# 定义保存模型的函数
def save_model(agent, agent_id, epoch, save_dir="saved_models"):
    os.makedirs(save_dir, exist_ok=True)  # 创建保存目录
    model_filename = f"agent_{agent_id}_epoch_{epoch}.pth"
    model_path = os.path.join(save_dir, model_filename)
    
    # 保存模型的state_dict
    torch.save(agent.model.state_dict(), model_path)
    print(f"Model saved to {model_path}")

In [ ]:

device = 'cuda'
# 创建每个代理时传入设备
agents = [
    
    Agent(
        model=neural_model,
        optimizer=optim.Adam(neural_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
    Agent(
        model=lstm_model,
        optimizer=optim.Adam(lstm_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
    Agent(
        model=mlp_model,
        optimizer=optim.Adam(mlp_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
    Agent(
        model=gru_model,
        optimizer=optim.Adam(gru_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
  
    Agent(
        model=transformer_model,
        optimizer=optim.Adam(transformer_model.parameters(), lr=0.1),
        loss_fn=nn.CrossEntropyLoss(),
        device=device
    ),
]

# Main loop for training, fusion, feedback, and optimization

train_losses = []
val_losses = []
accuracy_scores = []
agent_outputs = []


In [ ]:
def shap_predict_wrapper(agent, X):
    # 确保输入是正确的
    X = torch.tensor(X, dtype=torch.float32).to(device)
    return agent.model(X).detach().cpu().numpy()  # 返回的是一个 numpy 数组
num_epochs=10

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    # 记录loss
    agent_outputs = []
    for batch_idx, (data, target) in enumerate(train_loader):
        losses = []
        for i, agent in enumerate(agents):
            if i==0:
                print(f"Training Agent {i + 1}")
                agent_loss = []
                #print(data)
                #print(target)
                loss = agent.train_NeuralODE(data, target)  # 改为单批次训练
                agent_loss.append(loss)  # 将每个batch的loss存入列表
                if batch_idx % 100 == 0:  # 每100个batch打印一次
                    print(f"Batch {batch_idx}, Loss: {loss}")
                # 将当前agent的所有loss加入到全局loss记录中
                losses.append(np.mean(agent_loss))  # 记录当前agent每个epoch的loss平均值
                agent_outputs.append(agent.predict_NeuralODE(X_test))
                # 每个epoch结束后保存模型
                #save_model(agent, i, 1)    
            else:
                print(f"Training Agent {i + 1}")
                agent_loss = []
                #print(f"Batch {batch_idx}, Data: {data}")  # 打印数据结构
                loss = agent.train(data, target)  # 改为单批次训练
                agent_loss.append(loss)  # 将每个batch的loss存入列表
                if batch_idx % 100 == 0:  # 每100个batch打印一次
                    print(f"Batch {batch_idx}, Loss: {loss}")
                # 将当前agent的所有loss加入到全局loss记录中
                losses.append(np.mean(agent_loss))  # 记录当前agent每个epoch的loss平均值
                # Generate predictions for FB distribution
                agent_outputs.append(agent.predict(X_test))
                # 每个epoch结束后保存模型
                #save_model(agent, i, 1)     
    # 将 numpy 数组改为 torch 张量，并且确保这些张量有梯度
    fb_distribution = torch.mean(torch.tensor(agent_outputs, dtype=torch.float32), dim=0, keepdim=False)  # 使用 torch.mean 计算均值
    # 进一步训练基于 FB 分布反馈
    for i, agent in enumerate(agents):
        if i >= len(agent_outputs):  # 防止越界访问
            print(f"Warning: agent_outputs does not have an output for agent {i + 1}")
            continue  # 跳过当前 agent
        # 确保 agent_outputs 和 fb_distribution 是 Torch 张量并且需要梯度
        raw_output = torch.tensor(agent_outputs[i], dtype=torch.float32, requires_grad=True)  # 使 raw_output 具有梯度
        fb_distribution_tensor = fb_distribution  # 此时 fb_distribution 已经是 Torch 张量，继续使用
        # 使用损失函数计算损失
        fb_loss = agent.loss_fn(raw_output, fb_distribution_tensor)
        # 反向传播
        agent.optimizer.zero_grad()
        fb_loss.backward()  # 现在可以正常计算梯度
        agent.optimizer.step()
    # Shapley value calculation
    shapley_values = []
    for i, agent in enumerate(agents):
        # Randomly sample from X_train for Shapley calculation
        if(i==0):
            sample_idx = np.random.choice(len(X_train), size=100, replace=False)
            X_sample = X_train[sample_idx]
            explainer = shap.KernelExplainer(agent.predict_NeuralODE, X_sample)
            shap_values = explainer.shap_values(X_test)
            shapley_values.append(np.mean(shap_values))
        else:
            sample_idx = np.random.choice(len(X_train), size=100, replace=False)
            X_sample = X_train[sample_idx]
            explainer = shap.KernelExplainer(agent.predict, X_sample)
            shap_values = explainer.shap_values(X_test)
            shapley_values.append(np.mean(shap_values))
    # Reward and learning rate adjustment
    total_shapley = sum(shapley_values)
    for i, agent in enumerate(agents):
        reward = shapley_values[i] / total_shapley if total_shapley > 0 else 0
        agent.update_reward(reward)
        agent.adjust_weight()
        agent.adjust_learning_rate()

    # Display rewards
    for i, agent in enumerate(agents):
        print(f"Agent {i + 1} reward after iteration {epoch + 1}: {agent.reward}")
    # epoch结束
    print(f"Epoch {epoch+1} completed.")

In [ ]:
plt.figure(figsize=(10, 6))
for agent_name, losses in loss_curves.items():
    plt.plot(range(1, num_iterations + 1), losses, label=agent_name)

plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.title('Loss Curve for Each Agent')
plt.legend()
plt.grid()
plt.show()

In [ ]:
nvcc --version  